In [1]:
!pip install -r req.txt

In [2]:
!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cpu


In [3]:
!pip install --upgrade transformers

In [2]:
# Import all libs
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sklearn.metrics import (
    f1_score, recall_score, balanced_accuracy_score,
    matthews_corrcoef, precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [4]:
print(torch.mps.is_available()) #if using mac
# print(torch.cuda.is_available()) #if using GPU with cuda

True


In [5]:
# Authenticate with HuggingFace
import os
from dotenv import load_dotenv
from huggingface_hub import login

# from google.colab import userdata
# hugging_face_token = userdata.get("hf_token") #If using gg colab

load_dotenv() #If using VSCode
hugging_face_token = os.getenv("hf_token") #If using VSCode

login(token=hugging_face_token)

Load AG news datasets

In [23]:
label_map = {
    0: "world",
    1: "sports",
    2: "business",
    3: "sci/tech"
}
ag_news_imbalanced_data_10_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced.parquet")
balanced_data = pd.read_parquet("Data/ag_news/ag_news_train_balanced.parquet")
ag_news_imbalanced_data_5_to_1 = pd.read_parquet("Data/ag_news/ag_news_train_imbalanced_5_to_1_ratio.parquet")

balanced_data["label"] = balanced_data["label"].map(label_map)
ag_news_imbalanced_data_10_to_1["label"] = ag_news_imbalanced_data_10_to_1["label"].map(label_map)
ag_news_imbalanced_data_5_to_1["label"] = ag_news_imbalanced_data_5_to_1["label"].map(label_map)

# Shuffle the dataset
ag_news_imbalanced_data_10_to_1 = ag_news_imbalanced_data_10_to_1.sample(frac=1).reset_index(drop=True)
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)
ag_news_imbalanced_data_5_to_1 = ag_news_imbalanced_data_5_to_1.sample(frac=1).reset_index(drop=True)

# Testing
balanced_data



,text,label
0,"Electronics, Health Care Profit at Tyco #39;s ...",business
1,Mercent to help Guess sell on Amazon E-commerc...,sci/tech
2,"Crimson find the range PRINCETON, N.J. -- You'...",sports
3,"SUN 61, LIBERTY 51 Balanced Sun Knocks Liberty...",sports
4,Protective goal posts come into play at Clemso...,sports
...,...,...
1995,Zale Reports Wider Third-Quarter Loss (AP) AP ...,business
1996,Europe probe arrives at the Moon The Smart 1 l...,sci/tech
1997,Pentagon Blocks Access to Site for Overseas Vo...,sci/tech
1998,Saturday qualifying at Japanese GP postponed d...,sports


Load toxic text dataset (The dataset is already imbalanced)

- Rename the columns to have the same names as other datasets
- Label = 0 --> Not toxic
- Label = 1 --> Toxic
- Map label column into words

In [ ]:
import math

toxic_label_map = {
    0: "nontoxic",
    1: "toxic"
}

toxic_text = pd.read_csv("Data/toxic_text/train.csv")
toxic_text = toxic_text[["comment_text", "toxic"]]
toxic_text = toxic_text.rename(columns={"comment_text": "text", "toxic": "label"})
toxic_text["label"] = toxic_text["label"].map(toxic_label_map)

# REDUCE THE SIZE OF THE DATASET HERE


# Get the imbalance ratio of the dataset
major = toxic_text[toxic_text['label'] == 'nontoxic']["label"].value_counts().iloc[0]
minor = toxic_text[toxic_text['label'] == 'toxic']["label"].value_counts().iloc[0]

gcd = math.gcd(major, minor)
major_ratio = major // gcd
minor_ratio = minor // gcd

print(f"=== THIS DATASET HAS A CLASS RATIO OF: {major_ratio}:{minor_ratio} ===")


=== THIS DATASET HAS A CLASS RATIO OF: 144277:15294 ===


(159571, 2)

Load twitter emotion type dataset (This is also imbalanced)

In [ ]:
emotion_map = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

emotion_df = pd.read_parquet("Data/twit/twitter_emotion.parquet")

# REDUCE THE SIZE OF THE DATASET HERE

label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64

In [ ]:
def build_prompt(df, text, label_map, shots_per_class=None):
    """
    Function to construct an instruction for the LLM

    Args:
        text (str): The text of the data

    Returns:
        prompt (str): The constructed prompt for the LLM
    """
    prompt = (
        f"You are a powerful, precise, and helpful assistant that classifies text into well-defined categories, NO MATTER THE CONTEXT."
        f" IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: {', '.join(list(label_map.values()))}."
        f" Respond with exactly one word: the single best category."
        f" Do not explain your choice, provide reasoning, or output anything else."
        f" Learn from these examples to understand context and edge cases: "

    )

    few_shots_example = [
        {"text": "Macedonian Prime Minister Kostov Resigns Parliament will officially announce Prime Minister Hari Kostov #39;s resignation during a meeting scheduled for Thursday, launching a ten-day period during which President Branko Crvenkovski should give the mandate for forming a new government.", 'label': 'business'},
        {'text': 'Trinidad climbs off canvas to keep title options open Felix Trinidad returned to the ring after more than two years to score a thrilling eighth-round stoppage of Ricardo Mayorga at New York #39;s Madison Square Garden in a non- title bout being described as one of the fights of the year.', 'label': 'sports'},
        {'text': 'Hungary #39;s ruling Socialist Party dumps PM BUDAPEST, Aug. 19 (Xinhuanet) -- Hungary #39;s ruling Socialist Party said Thursday that it accepted Prime Minister Peter Medgyessy #39;s resignation and has named a candidate for the post.', 'label': 'world'},
        {'text': 'Europe probe arrives at the Moon The Smart 1 lunar probe has entered into orbit around the Moon, the first ever European mission to do so. ', 'label': 'sci/tech'}
    ]

    prompt += "\n\n"
    for ex in few_shots_example:
        prompt += f"Review: \"{ex['text']}\"\nCategory: {ex['label']}\n\n"
    prompt += f"Review: \"{text}\"\nCategory:" #Leave Category here blank since we want the LLM to generate text
    return prompt


# Testing function
print(build_prompt("Astros Rally Past the Giants With one swing of the bat, Lance Berkman revived the Houston Astros' playoff hopes - and gave the Los Angeles Dodgers a much-needed reprieve. Berkman hit a three-run homer off Dustin Hermanson, highlighting a five-run ninth inning that sent Houston to a 7-3 win over San Francisco on Thursday night..."))

You are a powerful, precise, and helpful assistant that classifies text into well-defined categories. IMPORTANT: CHOOSE ONE WORD FROM THESE CATEGORIES: 'business', 'world', 'sci/tech', 'sports'. Respond with exactly one word: the single best category. Do not explain your choice, provide reasoning, or output anything else. Learn from these examples to understand context and edge cases: 

Review: "Macedonian Prime Minister Kostov Resigns Parliament will officially announce Prime Minister Hari Kostov #39;s resignation during a meeting scheduled for Thursday, launching a ten-day period during which President Branko Crvenkovski should give the mandate for forming a new government."
Category: business

Review: "Trinidad climbs off canvas to keep title options open Felix Trinidad returned to the ring after more than two years to score a thrilling eighth-round stoppage of Ricardo Mayorga at New York #39;s Madison Square Garden in a non- title bout being described as one of the fights of the ye

In [63]:
# Move model to mps (If using Mac)
# if torch.backends.mps.is_available():
#     model.to('mps')

# model.eval()

In [8]:
def clean_time(time):
  """
  Function to clean the time into prettier format, returns the better format of time
  """
  if time <= 60:
    return f"{time} seconds."

  minutes = time // 60
  remain_sec = time - minutes * 60
  return f"{minutes} minutes, {remain_sec:.2f} seconds."


In [ ]:

from tqdm.auto import tqdm
from transformers import pipeline, logging
from time import time


# Load model


# CREATE A FUNCTION TO RUN CLASSFICATION
def classify(model, df, batch_size):
    """
    Function to run classification with different number of shots

    Args:
        model (str): name of the model
        tokenizer
        df (pd.DataFrame): the pandas dataframe
        batch_size (int): batch size per run

    Returns:
        None
    """
    # Initiate a pipeline for each dataset
    # USE text2text-generation for the gemma model
    # USE text-generation for the others, or text-classification
    # USE fill-mask for distillbert
    pipe = pipeline("text-generation", model=model, dtype=torch.float16)
    logging.set_verbosity_error()

    # Generate prompts for all rows
    prompts = [build_prompt(text) for text in df["text"]]

    # Run the pipeline for each row
    pred_arr = []
    start_time = time()

    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size] #slices a sublist of prompts
        results = pipe(batch, max_new_tokens=3, do_sample=False)
        for prompt, res in zip(batch, results):
            pred = res[0]['generated_text'][len(prompt):].strip().split()
            # print(f"Real value: {df["label"]}")
            # print(f"Predicted value: {pred}")
            pred_arr.append(pred) #Use pred[0] for tiiuae/falcon-rw-1b
    end_time = time()

    total_time = clean_time(end_time - start_time)

    print("Total running time is " + total_time)
    df["llm_prediction"] = pred_arr
    print("Predictions have been added to the dataframe")



NOW FOCUSING ON QWEN2.5 INSTRUCT MODEDLS

In [ ]:

model = "Qwen/Qwen2.5-0.5B-Instruct" #Could be changed later for more evals
# microsoft/phi-2



# Run with 3 datasets
bs = 8

# classify(
#     model=model,
#     df=ag_news_imbalanced_data_10_to_1,
#     batch_size=bs
# )

# classify(
#     model=model,
#     df=ag_news_imbalanced_data_5_to_1,
#     batch_size=bs
# )

classify(
    model=model,
    df=balanced_data,
    batch_size=bs
)

Total running time is 19.0 minutes, 17.07 seconds.
Predictions have been added to the dataframe


In [19]:
def evaluate_model(df, true_col="label", pred_col='llm_prediction'):
  df[true_col] = df[true_col].str.lower().str.strip()
  df[pred_col] = df[pred_col].str.lower().str.strip()

  correct_pred = (df[true_col] == df[pred_col]).sum()
  total = len(df)
  accuracy = correct_pred / total

  print(f"\n✅ Accuracy: {accuracy*100:.2f}% ({correct_pred}/{total} correct)")

In [ ]:
evaluate_model(balanced_data)
evaluate_model(ag_news_imbalanced_data_10_to_1)
evaluate_model(ag_news_imbalanced_data_5_to_1)


✅ Accuracy: 45.00% (900/2000 correct)

✅ Accuracy: 59.75% (1195/2000 correct)

✅ Accuracy: 64.84% (1298/2002 correct)


In [ ]:
# Check LLM's output to see if it has correctly generated the word in the categories
ag_news_imbalanced_data_10_to_1["llm_prediction"]